In [1]:
%pwd

'/home/arch_aditya/Desktop/text-summarizer/notebooks'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/arch_aditya/Desktop/text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Entity for dataIngestion
@dataclass(frozen=True)
class DataIngestionConfig :
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager :
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_url = config.source_url,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
          )

          return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion :
    def __init__ (self , config:DataIngestionConfig) :
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , header = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} to {self.config.local_data_file}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory
        function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try :
    config = ConfigurationManager()
    print("done")
    data_ingestion_config = config.get_data_ingestion_config()
    print("done")
    data_ingestion = DataIngestion(config = data_ingestion_config)
    print("done")
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    logger.info("Data ingestion completed successfully")
except Exception as e:
    logger.error(e)
    raise e

[2024-03-17 02:17:14,414 - INFO - common - 28 - yaml file : config/config.yaml loaded successfully]
[2024-03-17 02:17:14,418 - INFO - common - 28 - yaml file : params.yaml loaded successfully]
[2024-03-17 02:17:14,421 - INFO - common - 57 - Created directory artifacts]
done
[2024-03-17 02:17:14,423 - INFO - common - 57 - Created directory artifacts/data_ingestion]
done
done
[2024-03-17 02:17:17,735 - INFO - 291430067 - 11 - Downloaded artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip]
[2024-03-17 02:17:17,873 - INFO - 1902062555 - 10 - Data ingestion completed successfully]
